In [45]:
import pandas as pd
import os

DATA_PATH = "/vol/bitbucket/jg2619/augmenting_llms/augmented_data_pipeline/data/train/curated"
FILE_NAME = "train_short.csv"
experiment_names = ["LLAMA2_llama-round2", "GPTJ_med_set"]

dfs = []

for experiment_name in experiment_names:
    df = pd.read_csv(os.path.join(DATA_PATH, experiment_name, FILE_NAME))
    df["experiment_name"] = experiment_name
    dfs.append(df)

In [46]:
# Number of rows of each tool_name for each experiment
df = pd.concat(dfs)

print(df.groupby(["experiment_name", "tool_name"]).agg({"tool_name": "count", "relevance": ["mean", "max", "min"]}))

                               tool_name relevance                     
                                   count      mean        max       min
experiment_name     tool_name                                          
GPTJ_med_set        Calculator      3000 -0.537919   2.961153 -0.927469
                    Calendar        1500 -0.082624   2.174629 -0.400528
                    WikiSearch      4950 -0.001672   4.248432 -0.462925
LLAMA2_llama-round2 Calculator      2843 -0.028793   6.044215 -0.487642
                    Calendar        3000 -1.731007   6.444887 -2.571811
                    WikiSearch      4685 -0.827435  25.095109 -2.242239


In [47]:
# Check how many urls appear uniquely in each experiment and how many appear in both
df2 = pd.concat(dfs)
print(f"Len of prefiltering {len(df2)}")
count = 0
for df in dfs:
    print(df["url"].nunique())
    count += df["url"].nunique()
print(df2["url"].nunique())

print(f"Diff: {count - df2['url'].nunique()}")

df0_urls = set(dfs[0]["url"].values)

dfs[1]["duplicate_url"] = False

for i, row in dfs[1].iterrows():
    if row["url"] in df0_urls:
        dfs[1].loc[i, "duplicate_url"] = True

print(len(dfs[1][dfs[1]["duplicate_url"]==True]))


Len of prefiltering 19978
5796
4600
9898
Diff: 498
961


In [58]:
dfs2 = [dfs[0], dfs[1][dfs[1].duplicate_url == False]]

df = pd.concat(dfs2)

len(df)

# delete column called duplicate_urls
del df["duplicate_url"]

In [60]:
# Save to csv:
df.to_csv(os.path.join(DATA_PATH, "LLAMA2_GPTJ_merged_2", "Calculator_train_short.csv"), index=False)